In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import numpy as np
from collections import defaultdict
import random

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


# ==================== ResNet Building Blocks ====================
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * self.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * self.expansion,
                         kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * self.expansion)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNetEmbedding(nn.Module):
    def __init__(self, block, num_blocks, embedding_dim=128, dropout=0.1):
        super(ResNetEmbedding, self).__init__()
        self.in_channels = 64
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.dropout = nn.Dropout2d(dropout)
        
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, embedding_dim)
        self.bn_emb = nn.BatchNorm1d(embedding_dim)
        
    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.dropout(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        out = self.bn_emb(out)
        return out


def ResNet18Embedding(embedding_dim=128, dropout=0.1):
    return ResNetEmbedding(BasicBlock, [2, 2, 2, 2], embedding_dim, dropout)


# ==================== Triplet Network ====================
class TripletNetwork(nn.Module):
    def __init__(self, embedding_net):
        super(TripletNetwork, self).__init__()
        self.embedding_net = embedding_net
        
    def forward(self, anchor, positive, negative):
        anchor_embedding = self.embedding_net(anchor)
        positive_embedding = self.embedding_net(positive)
        negative_embedding = self.embedding_net(negative)
        return anchor_embedding, positive_embedding, negative_embedding
    
    def get_embedding(self, x):
        return self.embedding_net(x)


# ==================== CORRECTED A-SCTL Loss ====================
class ASCTLLoss(nn.Module):
    def __init__(self, m1=0.01, m2=1.0, lambda_min=0.1, lambda_max=0.9):
        super(ASCTLLoss, self).__init__()
        self.m1 = m1  # Paper's value: 0.01
        self.m2 = m2  # Paper's value: 1.0
        self.lambda_min = lambda_min
        self.lambda_max = lambda_max
        self.lambda_k = 0.5
        
    def forward(self, anchor, positive, negative):
        # Squared Euclidean distances
        pos_dist = torch.sum((anchor - positive) ** 2, dim=1)
        neg_dist = torch.sum((anchor - negative) ** 2, dim=1)
        
        # Losses per paper
        intra_loss = torch.clamp(pos_dist - self.m1, min=0.0)
        inter_loss = torch.clamp(self.m2 - neg_dist, min=0.0)
        
        intra_loss_mean = torch.mean(intra_loss)
        inter_loss_mean = torch.mean(inter_loss)
        
        # Adaptive lambda - more responsive than before
        total_loss = intra_loss_mean.item() + inter_loss_mean.item()
        if total_loss > 1e-8:
            new_lambda = intra_loss_mean.item() / total_loss
            # Reduced smoothing for faster adaptation
            self.lambda_k = 0.6 * self.lambda_k + 0.4 * new_lambda
            self.lambda_k = max(self.lambda_min, min(self.lambda_max, self.lambda_k))
        
        loss = (1 - self.lambda_k) * intra_loss_mean + self.lambda_k * inter_loss_mean
        
        return loss, intra_loss_mean, inter_loss_mean, self.lambda_k


# ==================== Enhanced Triplet Dataset with Hard Mining ====================
class TripletDataset(Dataset):
    def __init__(self, dataset, train=True):
        self.dataset = dataset
        self.train = train
        self.labels = np.array([label for _, label in dataset])
        self.embeddings = None
        
        self.label_to_indices = defaultdict(list)
        for idx, label in enumerate(self.labels):
            self.label_to_indices[label].append(idx)
        
        self.labels_set = set(self.labels)
        
    def update_embeddings(self, embeddings):
        """Update embeddings for hard negative mining"""
        self.embeddings = embeddings.detach().cpu()
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, index):
        anchor_img, anchor_label = self.dataset[index]
        
        # Positive - same class, different sample
        positive_index = index
        while positive_index == index:
            positive_index = np.random.choice(self.label_to_indices[anchor_label])
        positive_img, _ = self.dataset[positive_index]
        
        # Hard/Semi-Hard Negative Mining
        if self.embeddings is not None and len(self.embeddings) > 0:
            anchor_emb = self.embeddings[index]
            
            # Collect candidates from all other classes
            negative_candidates = []
            for neg_label in (self.labels_set - {anchor_label}):
                neg_indices = self.label_to_indices[neg_label]
                # Sample subset for efficiency
                sampled = np.random.choice(neg_indices, min(10, len(neg_indices)), replace=False)
                negative_candidates.extend(sampled)
            
            if len(negative_candidates) > 0:
                neg_embs = self.embeddings[negative_candidates]
                distances = torch.sum((anchor_emb - neg_embs) ** 2, dim=1)
                
                # Semi-hard mining: select from top 30% hardest negatives
                # (not the absolute hardest to avoid outliers)
                k = max(1, int(len(distances) * 0.3))
                topk_indices = torch.topk(distances, k, largest=False)[1]
                selected_idx = topk_indices[torch.randint(len(topk_indices), (1,))]
                negative_index = negative_candidates[selected_idx.item()]
            else:
                # Fallback to random
                negative_label = np.random.choice(list(self.labels_set - {anchor_label}))
                negative_index = np.random.choice(self.label_to_indices[negative_label])
        else:
            # Random sampling (used in early epochs)
            negative_label = np.random.choice(list(self.labels_set - {anchor_label}))
            negative_index = np.random.choice(self.label_to_indices[negative_label])
        
        negative_img, _ = self.dataset[negative_index]
        return anchor_img, positive_img, negative_img, anchor_label


# ==================== Improved Prediction Network ====================
class PredictionNetwork(nn.Module):
    def __init__(self, embedding_dim=128, num_classes=10):
        super(PredictionNetwork, self).__init__()
        self.fc1 = nn.Linear(embedding_dim, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.dropout1 = nn.Dropout(0.3)
        
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout2 = nn.Dropout(0.2)
        
        self.fc3 = nn.Linear(256, num_classes)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x


# ==================== Embedding Quality Metrics ====================
def compute_embedding_quality(model, dataloader):
    """Compute intra-class and inter-class distances"""
    model.eval()
    embeddings = []
    labels_list = []
    
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            emb = model.get_embedding(images)
            embeddings.append(emb.cpu())
            labels_list.append(labels)
    
    embeddings = torch.cat(embeddings)
    labels_list = torch.cat(labels_list)
    
    # Compute intra-class distance
    intra_distances = []
    for label in torch.unique(labels_list):
        mask = labels_list == label
        class_emb = embeddings[mask]
        if len(class_emb) > 1:
            dists = torch.cdist(class_emb, class_emb, p=2)
            intra_distances.append(dists[torch.triu(torch.ones_like(dists), diagonal=1) == 1].mean())
    
    # Compute inter-class distance
    inter_distances = []
    unique_labels = torch.unique(labels_list)
    for i, label1 in enumerate(unique_labels):
        for label2 in unique_labels[i+1:]:
            emb1 = embeddings[labels_list == label1]
            emb2 = embeddings[labels_list == label2]
            dists = torch.cdist(emb1, emb2, p=2)
            inter_distances.append(dists.mean())
    
    avg_intra = torch.tensor(intra_distances).mean().item() if intra_distances else 0
    avg_inter = torch.tensor(inter_distances).mean().item() if inter_distances else 0
    
    return avg_intra, avg_inter


# ==================== Training Functions ====================
def train_triplet_network(model, criterion, optimizer, train_loader, epoch):
    model.train()
    total_loss = 0
    total_intra = 0
    total_inter = 0
    lambda_values = []
    
    for batch_idx, (anchor, positive, negative, labels) in enumerate(train_loader):
        anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)
        
        optimizer.zero_grad()
        
        anchor_emb, positive_emb, negative_emb = model(anchor, positive, negative)
        
        loss, intra_loss, inter_loss, lambda_k = criterion(anchor_emb, positive_emb, negative_emb)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        total_loss += loss.item()
        total_intra += intra_loss.item()
        total_inter += inter_loss.item()
        lambda_values.append(lambda_k)
        
        if batch_idx % 100 == 0:
            print(f'Epoch {epoch} [{batch_idx}/{len(train_loader)}] '
                  f'Loss: {loss.item():.4f} | Intra: {intra_loss.item():.4f} | '
                  f'Inter: {inter_loss.item():.4f} | λ: {lambda_k:.3f}')
    
    avg_loss = total_loss / len(train_loader)
    avg_intra = total_intra / len(train_loader)
    avg_inter = total_inter / len(train_loader)
    avg_lambda = np.mean(lambda_values)
    
    return avg_loss, avg_intra, avg_inter, avg_lambda


def train_prediction_network(triplet_model, pred_network, optimizer, train_loader, epoch):
    triplet_model.eval()
    pred_network.train()
    total_loss = 0
    correct = 0
    total = 0
    
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        with torch.no_grad():
            embeddings = triplet_model.get_embedding(images)
        
        outputs = pred_network(embeddings)
        loss = criterion(outputs, labels)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(pred_network.parameters(), max_norm=1.0)
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    accuracy = 100. * correct / total
    avg_loss = total_loss / len(train_loader)
    
    return avg_loss, accuracy


def test_classification(triplet_model, pred_network, test_loader):
    triplet_model.eval()
    pred_network.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            embeddings = triplet_model.get_embedding(images)
            outputs = pred_network(embeddings)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    accuracy = 100. * correct / total
    return accuracy


# ==================== Main ====================
def main():
    # Optimized hyperparameters
    BATCH_SIZE = 128
    EMBEDDING_DIM = 128  # Paper's optimal value
    NUM_EPOCHS_TRIPLET = 80  # Increased for better convergence
    NUM_EPOCHS_PRED = 100
    LEARNING_RATE_TRIPLET = 0.001
    LEARNING_RATE_PRED = 0.1  # Higher initial LR with better scheduler
    
    # Stronger augmentation for better generalization
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])
    
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])
    
    print("Loading CIFAR-10 dataset...")
    train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                                  download=True, transform=transform_train)
    test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                                 download=True, transform=transform_test)
    
    triplet_train_dataset = TripletDataset(train_dataset, train=True)
    triplet_train_loader = DataLoader(triplet_train_dataset, batch_size=BATCH_SIZE,
                                      shuffle=True, num_workers=4, pin_memory=True)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                            shuffle=False, num_workers=4, pin_memory=True)
    
    print("\nInitializing ResNet-18 Triplet Network with A-SCTL...")
    embedding_net = ResNet18Embedding(embedding_dim=EMBEDDING_DIM, dropout=0.1).to(device)
    triplet_model = TripletNetwork(embedding_net).to(device)
    pred_network = PredictionNetwork(embedding_dim=EMBEDDING_DIM, num_classes=10).to(device)
    
    # CORRECTED: Paper's margin values
    triplet_criterion = ASCTLLoss(m1=0.01, m2=1.0, lambda_min=0.1, lambda_max=0.9)
    triplet_optimizer = optim.AdamW(triplet_model.parameters(), lr=LEARNING_RATE_TRIPLET, weight_decay=5e-4)
    triplet_scheduler = optim.lr_scheduler.CosineAnnealingLR(triplet_optimizer, T_max=NUM_EPOCHS_TRIPLET)
    
    print("\n" + "="*70)
    print("PHASE 1: Training Triplet Network with A-SCTL")
    print("="*70)
    
    best_triplet_loss = float('inf')
    for epoch in range(1, NUM_EPOCHS_TRIPLET + 1):
        avg_loss, avg_intra, avg_inter, avg_lambda = train_triplet_network(
            triplet_model, triplet_criterion, triplet_optimizer, triplet_train_loader, epoch
        )
        triplet_scheduler.step()
        
        print(f'Epoch {epoch:2d} | Loss: {avg_loss:.4f} | Intra: {avg_intra:.4f} | '
              f'Inter: {avg_inter:.4f} | λ: {avg_lambda:.4f}')
        
        # Update embeddings for hard mining every 10 epochs
        if epoch % 10 == 0:
            print("Updating embeddings for hard negative mining...")
            triplet_model.eval()
            all_embeddings = []
            with torch.no_grad():
                for images, _ in train_loader:
                    images = images.to(device)
                    emb = triplet_model.get_embedding(images)
                    all_embeddings.append(emb.cpu())
            all_embeddings = torch.cat(all_embeddings)
            triplet_train_dataset.update_embeddings(all_embeddings)
            print(f"✓ Embeddings updated (epoch {epoch})")
        
        if avg_loss < best_triplet_loss:
            best_triplet_loss = avg_loss
            torch.save(triplet_model.state_dict(), 'best_triplet_embeddings.pth')
    
    # Evaluate embedding quality
    print("\n" + "="*70)
    print("Evaluating Embedding Quality")
    print("="*70)
    avg_intra, avg_inter = compute_embedding_quality(triplet_model, test_loader)
    separation_ratio = avg_inter / avg_intra if avg_intra > 0 else 0
    print(f"Average Intra-class Distance: {avg_intra:.4f}")
    print(f"Average Inter-class Distance: {avg_inter:.4f}")
    print(f"Separation Ratio (Inter/Intra): {separation_ratio:.4f}")
    
    if separation_ratio < 2.5:
        print("⚠️  Warning: Separation ratio is low. Consider training longer.")
    else:
        print("✓ Good embedding separation achieved!")
    
    print("\n" + "="*70)
    print("PHASE 2: Training Prediction Network")
    print("="*70)
    
    pred_optimizer = optim.SGD(pred_network.parameters(), lr=LEARNING_RATE_PRED, 
                               momentum=0.9, weight_decay=5e-4, nesterov=True)
    pred_scheduler = optim.lr_scheduler.OneCycleLR(
        pred_optimizer, max_lr=LEARNING_RATE_PRED, 
        epochs=NUM_EPOCHS_PRED, steps_per_epoch=len(train_loader),
        pct_start=0.3, anneal_strategy='cos'
    )
    
    best_test_acc = 0
    best_epoch = 0
    patience = 0
    max_patience = 20
    
    for epoch in range(1, NUM_EPOCHS_PRED + 1):
        train_loss, train_acc = train_prediction_network(
            triplet_model, pred_network, pred_optimizer, train_loader, epoch
        )
        
        # Step scheduler per batch
        if epoch == 1:
            for _ in range(len(train_loader)):
                pred_scheduler.step()
        
        test_acc = test_classification(triplet_model, pred_network, test_loader)
        
        current_lr = pred_optimizer.param_groups[0]['lr']
        print(f'Epoch {epoch:3d} | LR: {current_lr:.5f} | Train Loss: {train_loss:.4f} | '
              f'Train Acc: {train_acc:.2f}% | Test Acc: {test_acc:.2f}%', end='')
        
        if test_acc > best_test_acc:
            best_test_acc = test_acc
            best_epoch = epoch
            patience = 0
            torch.save({
                'epoch': epoch,
                'triplet_model': triplet_model.state_dict(),
                'pred_network': pred_network.state_dict(),
                'test_acc': test_acc,
                'optimizer': pred_optimizer.state_dict(),
            }, 'best_optimized_model.pth')
            print(f' ✓ NEW BEST!')
        else:
            patience += 1
            print()
        
        # Early stopping
        if patience >= max_patience:
            print(f"\nEarly stopping triggered at epoch {epoch} (patience={max_patience})")
            break
    
    print("\n" + "="*70)
    print("FINAL RESULTS")
    print("="*70)
    print(f"Best Test Accuracy: {best_test_acc:.2f}% (achieved at epoch {best_epoch})")
    print(f"Embedding Separation Ratio: {separation_ratio:.2f}x")
    
    if best_test_acc >= 85:
        print(f"\n🎉 EXCELLENT! Target exceeded ({best_test_acc:.2f}% >= 85%)")
    elif best_test_acc >= 80:
        print(f"\n✓ SUCCESS! Target achieved ({best_test_acc:.2f}% >= 80%)")
    elif best_test_acc >= 75:
        print(f"\n✓ GOOD! Close to target ({best_test_acc:.2f}%)")
    else:
        print(f"\nCurrent: {best_test_acc:.2f}%, Target: 80.00%")
        print(f"Gap to target: {80.00 - best_test_acc:.2f}%")
    
    return triplet_model, pred_network, best_test_acc


if __name__ == "__main__":
    print("\n" + "="*70)
    print("Optimized ResNet-18 Triplet Network with A-SCTL")
    print("Target: 80-85% Test Accuracy on CIFAR-10")
    print("="*70 + "\n")
    
    triplet_model, pred_network, accuracy = main()
    
    print("\n" + "="*70)
    print("Training completed! Best model saved to 'best_optimized_model.pth'")
    print("="*70)

Using device: cuda

Optimized ResNet-18 Triplet Network with A-SCTL
Target: 80-85% Test Accuracy on CIFAR-10

Loading CIFAR-10 dataset...


100%|██████████| 170M/170M [00:02<00:00, 66.3MB/s] 



Initializing ResNet-18 Triplet Network with A-SCTL...

PHASE 1: Training Triplet Network with A-SCTL
Epoch 1 [0/391] Loss: 71.8246 | Intra: 239.4152 | Inter: 0.0000 | λ: 0.700
Epoch 1 [100/391] Loss: 8.9577 | Intra: 89.4097 | Inter: 0.0186 | λ: 0.900
Epoch 1 [200/391] Loss: 4.8185 | Intra: 47.8333 | Inter: 0.0391 | λ: 0.900
Epoch 1 [300/391] Loss: 4.6714 | Intra: 46.0021 | Inter: 0.0791 | λ: 0.900
Epoch  1 | Loss: 7.8550 | Intra: 76.3206 | Inter: 0.0502 | λ: 0.8993
Epoch 2 [0/391] Loss: 2.4008 | Intra: 23.4455 | Inter: 0.0625 | λ: 0.900
Epoch 2 [100/391] Loss: 2.1799 | Intra: 20.8377 | Inter: 0.1068 | λ: 0.900
Epoch 2 [200/391] Loss: 0.9128 | Intra: 8.3856 | Inter: 0.0825 | λ: 0.900
Epoch 2 [300/391] Loss: 0.3274 | Intra: 2.6171 | Inter: 0.0730 | λ: 0.900
Epoch  2 | Loss: 1.2099 | Intra: 11.4800 | Inter: 0.0687 | λ: 0.9000
Epoch 3 [0/391] Loss: 0.2109 | Intra: 1.4012 | Inter: 0.0786 | λ: 0.900
Epoch 3 [100/391] Loss: 0.1878 | Intra: 1.3275 | Inter: 0.0612 | λ: 0.900
Epoch 3 [200/391] 